In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests
import time
import uuid

In [3]:
from package.keypair_management import KeyPairManagement
from package.jwt_management import ClientJWTManagement, ServerJWTManagement, JWTValidation
from package.database_management import DBManagement
from package.database_management import token_store
from package.utils import config

In [4]:
server = ServerJWTManagement()

In [5]:
db = DBManagement(db_path='./db/token_store.db')
db.delete_database()
tables = {
    config.REFRESH_TOKEN_JTI:"(jti STRING, token STRING, client_id STRING, exp INTEGER, active BOOLEAN, remark STRING)",
    config.ACCESS_TOKEN_JTI:"(jti STRING, token STRING, client_id STRING, exp INTEGER, active BOOLEAN, remark STRING)",
    config.DPOP_PROOF_JTI: "(jti STRING)"
}
for table, query in tables.items():
    db.create_database(table=table, schemas=query)

CREATE TABLE refresh_token_jti (jti STRING, token STRING, client_id STRING, exp INTEGER, active BOOLEAN, remark STRING)
CREATE TABLE access_token_jti (jti STRING, token STRING, client_id STRING, exp INTEGER, active BOOLEAN, remark STRING)
CREATE TABLE dpop_proof_jti (jti STRING)


In [6]:
BASE_URL = "http://localhost:8000"

In [7]:
km = KeyPairManagement(directory='./client_keypair')
skm = KeyPairManagement(directory='./server_keypair')
km.generate_keypairs()

Directory './client_keypair' already exists.


In [8]:
requests.get(f"{BASE_URL}/health").json()

{'response': 'Alive!'}

In [9]:
client = ClientJWTManagement()
headers = client.create_headers(public_key=km.load_public_key_from_pem())
payload = client.create_payload(
    jti=str(uuid.uuid4()),
    iat=int(time.time()),
    exp=config.DPOP_TOKEN_TIME,
    method="GET", uri=f"{BASE_URL}/authorizer/token", data={"client_id": "5555"}
)

In [10]:
headers

{'typ': 'dpop+jwt',
 'alg': 'RS256',
 'jwk': {'kty': 'RSA',
  'n': '2CaIvPf4-yMSOjugPeINrxxyG8ufS6nhswptU5akRsStoq-dfJjkdRMb7VFgurNIbMf7CSknF5w34-ABQhssH1y3GERrCAwdslh68pebGvlxinl3ZZtIbx2GUGfdl6JT388gPaDBJzkU0WsstHkkV-mmg_M2V5Vt-dV6Iu9y0JOvmpTOmKx8EHU3vqvoFa0kg0RLwScR_Ui3Y2DFQ3Ea9DJ5CU9339g-ZuI0A75tI5l_WOwHuWPzkY4FuxpMUGaRG4zkAW0gO4DkW-7jis7Y89xU_vGYVz-Z5O22ZQJwwBJ9lP38bnaGidGB-yEPMd5iaJKjbldrCAYf18yKni9dPw',
  'e': 'AQAB',
  'kid': 'pmZ2jvzEuAkIlFZ7dP8mbwYx2EYdmhAjFn2nKx427l8'}}

In [11]:
payload

{'htm': 'GET',
 'htu': 'http://localhost:8000/authorizer/token',
 'iat': 1732936112,
 'exp': 30,
 'jti': 'bade7180-c45a-4ffd-9de0-2ab4855f642b',
 'client_id': '5555'}

# Scenarios

In [12]:
client = ClientJWTManagement()
# server = ServerJWTManagement()
def sing_signature(jti:str, iat:int, exp:int, method:str, url:str, data:dict, public_key:bytes, private_key:bytes):
    headers = client.create_headers(public_key=public_key)
    payload = client.create_payload(
        jti=jti,
        iat=iat,
        exp=iat+exp,
        method=method, uri=url, data=data
    )
    client_signature = client.sign_by_client(headers=headers, payload=payload, private_key=private_key)
    return client_signature

def pack_headers(client_signature, access_token=None):
    headers = {
        "DPoP": client_signature
    }
    if access_token is not None:
        headers.update({"Authorization": f"DPoP {access_token}"})
    return headers

def test_get_endpoint(url, client_signature, access_token=None):
    headers = pack_headers(client_signature, access_token)
    response = requests.get(url, headers=headers)
    print("status code: ", response.status_code)
    response = response.json()
    return response

def test_post_endpoint(url, payload, client_signature, access_token=None):
    headers = pack_headers(client_signature, access_token)
    response = requests.post(url, headers=headers, json=payload)
    response = response.json()
    return response

def refresh_database(db, config):
    for table in [config.DPOP_PROOF_JTI, config.ACCESS_TOKEN_JTI, config.REFRESH_TOKEN_JTI]:
        db.execute(f"DELETE FROM {table};", [])

## Happy Path

In [13]:
refresh_database(db, config)

In [14]:
signature = sing_signature(
    jti=str(uuid.uuid4()), iat=int(time.time()), exp=config.DPOP_TOKEN_TIME, 
    method="GET", url=f"{BASE_URL}/authorizer/token", data={"client_id": "5555"},
    public_key=km.load_public_key_from_pem(),
    private_key=km.load_private_key_from_pem(),
)
response = test_get_endpoint(url=f"{BASE_URL}/authorizer/token", client_signature=signature, access_token=None)
access_token = response['access_token']
refresh_token = response['refresh_token']
# response

status code:  200


In [15]:
signature = sing_signature(
    jti=str(uuid.uuid4()), iat=int(time.time()), exp=config.DPOP_TOKEN_TIME, 
    method="GET", url=f"{BASE_URL}/resource/history", data={"client_id": "5555"},
    public_key=km.load_public_key_from_pem(),
    private_key=km.load_private_key_from_pem(),
)
response = test_get_endpoint(url=f"{BASE_URL}/resource/history", client_signature=signature, access_token=access_token)
response

status code:  200


{'response': 'you now get history'}

## DPoP expire

In [16]:
refresh_database(db, config)

In [17]:
signature = sing_signature(
    jti=str(uuid.uuid4()), iat=int(time.time()), exp=1, 
    method="GET", url=f"{BASE_URL}/authorizer/token", data={"client_id": "5555"},
    public_key=km.load_public_key_from_pem(),
    private_key=km.load_private_key_from_pem(),
)

time.sleep(2)

response = test_get_endpoint(url=f"{BASE_URL}/authorizer/token", client_signature=signature, access_token=None)
response

status code:  401


{'detail': 'DPoP Proof JWT error: Signature has expired.'}

## DPoP tampered

In [18]:
refresh_database(db, config)

In [19]:
# signature = sing_signature(dpop_life_time=60*5, method="GET", url=f"{BASE_URL}/authorizer/token", data={"client_id": "5555"})
signature = sing_signature(
    jti=str(uuid.uuid4()), iat=int(time.time()), exp=60*5, 
    method="GET", url=f"{BASE_URL}/authorizer/token", data={"client_id": "5555"},
    public_key=km.load_public_key_from_pem(),
    private_key=km.load_private_key_from_pem(),
)

In [20]:
tampered_signature = sing_signature(
    jti=str(uuid.uuid4()), iat=int(time.time()), exp=60*5, 
    method="GET", url=f"{BASE_URL}/authorizer/token", data={"client_id": "6666"},
    public_key=km.load_public_key_from_pem(),
    private_key=km.load_private_key_from_pem(),
)

In [21]:
headers, payload, signature = signature.split(".")
_, tampered_payload, _ = tampered_signature.split(".")

In [22]:
response = test_get_endpoint(url=f"{BASE_URL}/authorizer/token", client_signature=".".join([headers, tampered_payload, signature]), access_token=None)
response

status code:  400


{'detail': 'Error: General DPoP Proof JWT verification issue. Signature verification failed.'}

## Invalid claims: method and url

In [23]:
refresh_database(db, config)

In [24]:
signature = sing_signature(
    jti=str(uuid.uuid4()), iat=int(time.time()), exp=config.DPOP_TOKEN_TIME, 
    method="GET", url=f"{BASE_URL}/authorizer/token", data={"client_id": "5555"},
    public_key=km.load_public_key_from_pem(),
    private_key=km.load_private_key_from_pem(),
)
response = test_get_endpoint(url=f"{BASE_URL}/authorizer/token", client_signature=signature, access_token=None)
access_token = response['access_token']
refresh_token = response['refresh_token']
# response

status code:  200


In [25]:
signature = sing_signature(
    jti=str(uuid.uuid4()), iat=int(time.time()), exp=config.DPOP_TOKEN_TIME, 
    method="POST", url=f"{BASE_URL}/resource/history1", data={"client_id": "5555"},
    public_key=km.load_public_key_from_pem(),
    private_key=km.load_private_key_from_pem(),
)
response = test_get_endpoint(url=f"{BASE_URL}/resource/history", client_signature=signature, access_token=access_token)
response

status code:  400


{'detail': 'Invalid claim'}

## Invalid claims: method

In [26]:
refresh_database(db, config)

In [27]:
signature = sing_signature(
    jti=str(uuid.uuid4()), iat=int(time.time()), exp=config.DPOP_TOKEN_TIME, 
    method="GET", url=f"{BASE_URL}/authorizer/token", data={"client_id": "5555"},
    public_key=km.load_public_key_from_pem(),
    private_key=km.load_private_key_from_pem(),
)
response = test_get_endpoint(url=f"{BASE_URL}/authorizer/token", client_signature=signature, access_token=None)
access_token = response['access_token']
refresh_token = response['refresh_token']
# response

status code:  200


In [28]:
signature = sing_signature(
    jti=str(uuid.uuid4()), iat=int(time.time()), exp=config.DPOP_TOKEN_TIME, 
    method="POST", url=f"{BASE_URL}/resource/history", data={"client_id": "5555"},
    public_key=km.load_public_key_from_pem(),
    private_key=km.load_private_key_from_pem(),
)
response = test_get_endpoint(url=f"{BASE_URL}/resource/history", client_signature=signature, access_token=access_token)
response

status code:  400


{'detail': 'Invalid claim: method mismatched.'}

## Invalid claims: url

In [29]:
refresh_database(db, config)

In [30]:
signature = sing_signature(
    jti=str(uuid.uuid4()), iat=int(time.time()), exp=config.DPOP_TOKEN_TIME, 
    method="GET", url=f"{BASE_URL}/authorizer/token", data={"client_id": "5555"},
    public_key=km.load_public_key_from_pem(),
    private_key=km.load_private_key_from_pem(),
)
response = test_get_endpoint(url=f"{BASE_URL}/authorizer/token", client_signature=signature, access_token=None)
access_token = response['access_token']
refresh_token = response['refresh_token']
# response

status code:  200


In [31]:
signature = sing_signature(
    jti=str(uuid.uuid4()), iat=int(time.time()), exp=config.DPOP_TOKEN_TIME, 
    method="GET", url=f"{BASE_URL}/resource/history1", data={"client_id": "5555"},
    public_key=km.load_public_key_from_pem(),
    private_key=km.load_private_key_from_pem(),
)
response = test_get_endpoint(url=f"{BASE_URL}/resource/history", client_signature=signature, access_token=access_token)
response

status code:  400


{'detail': 'Invalid claim: url mismatched.'}

## Invalid jti: dpop

In [32]:
refresh_database(db, config)

In [33]:
jti = "thisisatest"

signature = sing_signature(
    jti=jti, iat=int(time.time()), exp=config.DPOP_TOKEN_TIME, 
    method="GET", url=f"{BASE_URL}/authorizer/token", data={"client_id": "5555"},
    public_key=km.load_public_key_from_pem(),
    private_key=km.load_private_key_from_pem(),
)
response = test_get_endpoint(url=f"{BASE_URL}/authorizer/token", client_signature=signature, access_token=None)
access_token = response['access_token']
refresh_token = response['refresh_token']
# response

status code:  200


In [34]:
signature = sing_signature(
    jti=jti, iat=int(time.time()), exp=config.DPOP_TOKEN_TIME, 
    method="GET", url=f"{BASE_URL}/resource/history", data={"client_id": "5555"},
    public_key=km.load_public_key_from_pem(),
    private_key=km.load_private_key_from_pem(),
)
response = test_get_endpoint(url=f"{BASE_URL}/resource/history", client_signature=signature, access_token=access_token)
response

status code:  400


{'detail': 'DPoP Replay detected.'}

## Invalid access token: tampered

In [35]:
refresh_database(db, config)

In [36]:
signature = sing_signature(
    jti=str(uuid.uuid4()), iat=int(time.time()), exp=config.DPOP_TOKEN_TIME, 
    method="GET", url=f"{BASE_URL}/authorizer/token", data={"client_id": "5555"},
    public_key=km.load_public_key_from_pem(),
    private_key=km.load_private_key_from_pem(),
)
response = test_get_endpoint(url=f"{BASE_URL}/authorizer/token", client_signature=signature, access_token=None)
access_token = response['access_token']
refresh_token = response['refresh_token']
# response

status code:  200


In [37]:
# tampered
signature = sing_signature(
    jti=str(uuid.uuid4()), iat=int(time.time()), exp=config.DPOP_TOKEN_TIME, 
    method="GET", url=f"{BASE_URL}/authorizer/token", data={"client_id": "6666"},
    public_key=km.load_public_key_from_pem(),
    private_key=km.load_private_key_from_pem(),
)
response = test_get_endpoint(url=f"{BASE_URL}/authorizer/token", client_signature=signature, access_token=None)
tampered_access_token = response['access_token']
tampered_refresh_token = response['refresh_token']
# response

status code:  200


In [38]:
h, p, s = access_token.split(".")
_, t_p, _ = tampered_access_token.split(".")
tampered_signature = ".".join([h,t_p,s])

signature = sing_signature(
    jti=str(uuid.uuid4()), iat=int(time.time()), exp=config.DPOP_TOKEN_TIME, 
    method="GET", url=f"{BASE_URL}/resource/history", data={"client_id": "5555"},
    public_key=km.load_public_key_from_pem(),
    private_key=km.load_private_key_from_pem(),
)
response = test_get_endpoint(url=f"{BASE_URL}/resource/history", client_signature=signature, access_token=tampered_signature)
response

status code:  401


{'detail': 'Unauthorize: access token tampered.'}

In [39]:
db.query(f"SELECT * FROM {config.ACCESS_TOKEN_JTI}")

,jti,token,client_id,exp,active,remark
0,188a22ec-0310-472c-b35e-2aeb47b46bb6,eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJpYXQiO...,5555,1732936455,True,valid
1,71b11d6b-f336-499d-a263-22b06ddd6411,eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJpYXQiO...,6666,1732936458,False,tampered


## Invalid access token: replayed

In [40]:
refresh_database(db, config)

In [41]:
# jti = str(uuid.uuid4())

signature = sing_signature(
    jti=str(uuid.uuid4()), iat=int(time.time()), exp=config.DPOP_TOKEN_TIME, 
    method="GET", url=f"{BASE_URL}/authorizer/token", data={"client_id": "5555"},
    public_key=km.load_public_key_from_pem(),
    private_key=km.load_private_key_from_pem(),
)
response = test_get_endpoint(url=f"{BASE_URL}/authorizer/token", client_signature=signature, access_token=None)
access_token = response['access_token']
refresh_token = response['refresh_token']
# response

status code:  200


In [42]:
_, p, s = access_token.split(".")
p = server.decode_from_jwt(p)
jti = p['jti']

In [43]:
db.query(f"SELECT * FROM {config.ACCESS_TOKEN_JTI} WHERE jti=='{jti}' and token=='{access_token}'")

,jti,token,client_id,exp,active,remark
0,c6cf48a5-ad33-4960-b52a-b2e7c74598e6,eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJpYXQiO...,5555,1732936465,True,valid


In [44]:
query = f"""UPDATE {config.ACCESS_TOKEN_JTI} SET active = ?, remark = ? WHERE jti=='{jti}' AND token=='{access_token}';"""

db.execute(query, (False, "test"))

In [45]:
db.query(f"SELECT * FROM {config.ACCESS_TOKEN_JTI} WHERE jti=='{jti}' and token=='{access_token}'")

,jti,token,client_id,exp,active,remark
0,c6cf48a5-ad33-4960-b52a-b2e7c74598e6,eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJpYXQiO...,5555,1732936465,False,test


In [46]:
db.query(f"SELECT * FROM {config.ACCESS_TOKEN_JTI}")

,jti,token,client_id,exp,active,remark
0,c6cf48a5-ad33-4960-b52a-b2e7c74598e6,eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJpYXQiO...,5555,1732936465,False,test


In [47]:
signature = sing_signature(
    jti=str(uuid.uuid4()), iat=int(time.time()), exp=config.DPOP_TOKEN_TIME, 
    method="GET", url=f"{BASE_URL}/resource/history", data={"client_id": "5555"},
    public_key=km.load_public_key_from_pem(),
    private_key=km.load_private_key_from_pem(),
)
response = test_get_endpoint(url=f"{BASE_URL}/resource/history", client_signature=signature, access_token=access_token)
response

status code:  401


{'detail': 'Unauthorize: access token replayed.'}

In [48]:
db.query(f"SELECT * FROM {config.ACCESS_TOKEN_JTI}")

,jti,token,client_id,exp,active,remark
0,c6cf48a5-ad33-4960-b52a-b2e7c74598e6,eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJpYXQiO...,5555,1732936465,False,replayed


In [49]:
db.query(f"SELECT * FROM {config.REFRESH_TOKEN_JTI}")

,jti,token,client_id,exp,active,remark
0,73b472cd-c052-4809-bd55-599d29e147c8,eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJpYXQiO...,5555,1733022565,True,valid


## Invalid access token: expired

In [50]:
refresh_database(db, config)

In [51]:
client_public = km.load_public_key_from_pem()
client_public_jwk = server.convert_pem_to_jwk(client_public)
thumbprint = server.encode_public_key_jwk_thumprint(client_public_jwk)

iat = int(time.time())
private_key = skm.load_private_key_from_pem()
client_id = "5555"
access_token_jti = str(uuid.uuid4())
exp = iat+1
access_token = server.generate_token(
    jti=access_token_jti,
    iat=iat, 
    exp=exp,
    private_key=private_key,
    algorithm=server.algorithm,
    data={
        "client_id": client_id, 
        "cnf":{"jkt": thumbprint}
    }
)
token_store.add_token(table=config.ACCESS_TOKEN_JTI, jti=access_token_jti, token=access_token, client_id=client_id, exp=exp, active=True, remark="valid")

time.sleep(1)

refresh_token_jti = str(uuid.uuid4())
exp = iat+config.REFRESH_TOKEN_TIME
refresh_token = server.generate_token(
    jti=refresh_token_jti,
    iat=iat, 
    exp=exp,
    private_key=private_key,
    algorithm=server.algorithm,
    data={
        "client_id":client_id
    }
)
token_store.add_token(table=config.REFRESH_TOKEN_JTI, jti=refresh_token_jti, token=refresh_token, client_id=client_id, exp=exp, active=True, remark="valid")

In [52]:
signature = sing_signature(
    jti=str(uuid.uuid4()), iat=int(time.time()), exp=config.DPOP_TOKEN_TIME, 
    method="GET", url=f"{BASE_URL}/resource/history", data={"client_id": "5555"},
    public_key=km.load_public_key_from_pem(),
    private_key=km.load_private_key_from_pem(),
)
response = test_get_endpoint(url=f"{BASE_URL}/resource/history", client_signature=signature, access_token=access_token)
response

status code:  401


{'detail': 'Unauthorize: access token expired.'}

In [53]:
db.query(f"SELECT * FROM {config.ACCESS_TOKEN_JTI}")

,jti,token,client_id,exp,active,remark
0,6243f93c-9cfb-4a7d-9401-8c80357bd605,eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJpYXQiO...,5555,1732936171,False,expired


In [54]:
db.query(f"SELECT * FROM {config.REFRESH_TOKEN_JTI}")

,jti,token,client_id,exp,active,remark
0,541fc527-81b5-4068-91bb-da7ab8b6573e,eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJpYXQiO...,5555,1733022570,True,valid


## Invalid access token: not exist

In [55]:
refresh_database(db, config)

In [56]:
client_public = km.load_public_key_from_pem()
client_public_jwk = server.convert_pem_to_jwk(client_public)
thumbprint = server.encode_public_key_jwk_thumprint(client_public_jwk)

iat = int(time.time())
private_key = skm.load_private_key_from_pem()
client_id = "5555"
access_token_jti = str(uuid.uuid4())
exp = iat+config.ACCESS_TOKEN_TIME
access_token = server.generate_token(
    jti=access_token_jti,
    iat=iat, 
    exp=exp,
    private_key=private_key,
    algorithm=server.algorithm,
    data={
        "client_id": client_id, 
        "cnf":{"jkt": thumbprint}
    }
)

In [57]:
signature = sing_signature(
    jti=str(uuid.uuid4()), iat=int(time.time()), exp=config.DPOP_TOKEN_TIME, 
    method="GET", url=f"{BASE_URL}/resource/history", data={"client_id": "5555"},
    public_key=km.load_public_key_from_pem(),
    private_key=km.load_private_key_from_pem(),
)
response = test_get_endpoint(url=f"{BASE_URL}/resource/history", client_signature=signature, access_token=access_token)
response

status code:  400


{'detail': 'access token inexist.'}

## Invalid refresh token: tampered

In [58]:
refresh_database(db, config)

In [59]:
signature = sing_signature(
    jti=str(uuid.uuid4()), iat=int(time.time()), exp=config.DPOP_TOKEN_TIME, 
    method="GET", url=f"{BASE_URL}/authorizer/token", data={"client_id": "5555"},
    public_key=km.load_public_key_from_pem(),
    private_key=km.load_private_key_from_pem(),
)
response = test_get_endpoint(url=f"{BASE_URL}/authorizer/token", client_signature=signature, access_token=None)
access_token = response['access_token']
refresh_token = response['refresh_token']
# response

status code:  200


In [60]:
refresh_token_jti = str(uuid.uuid4())
refresh_token_jti = ""
exp = iat+1
tampered_refresh_token = server.generate_token(
    jti=refresh_token_jti,
    iat=iat, 
    exp=exp,
    private_key=private_key,
    algorithm=server.algorithm,
    data={
        "client_id":"5555"
    }
)

In [61]:
h, p, s = refresh_token.split(".")
_, t_p, _ = tampered_refresh_token.split(".")
tampered_signature = ".".join([h,t_p,s])

signature = sing_signature(
    jti=str(uuid.uuid4()), iat=int(time.time()), exp=config.DPOP_TOKEN_TIME, 
    method="GET", url=f"{BASE_URL}/authorizer/refresh", data={"client_id": "5555"},
    public_key=km.load_public_key_from_pem(),
    private_key=km.load_private_key_from_pem(),
)
response = test_get_endpoint(url=f"{BASE_URL}/authorizer/refresh", client_signature=signature, access_token=tampered_signature)
response

status code:  403


{'detail': 'Unauthorize: refresh token tampered.'}

In [62]:
db.query(f"SELECT * FROM {config.ACCESS_TOKEN_JTI}")

,jti,token,client_id,exp,active,remark
0,2e006a71-422e-4d8f-aa2d-4912f8910a8f,eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJpYXQiO...,5555,1732936482,False,tampered


In [63]:
db.query(f"SELECT * FROM {config.REFRESH_TOKEN_JTI}")

,jti,token,client_id,exp,active,remark
0,8cbe54d1-54fb-4f43-aeed-706007ec32fe,eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJpYXQiO...,5555,1733022582,False,tampered


## Invalid refresh token: replayed

In [64]:
refresh_database(db, config)

In [65]:
signature = sing_signature(
    jti=str(uuid.uuid4()), iat=int(time.time()), exp=config.DPOP_TOKEN_TIME, 
    method="GET", url=f"{BASE_URL}/authorizer/token", data={"client_id": "5555"},
    public_key=km.load_public_key_from_pem(),
    private_key=km.load_private_key_from_pem(),
)
response = test_get_endpoint(url=f"{BASE_URL}/authorizer/token", client_signature=signature, access_token=None)
access_token = response['access_token']
refresh_token = response['refresh_token']
# response

status code:  200


In [66]:
_, p, s = refresh_token.split(".")
p = server.decode_from_jwt(p)
jti = p['jti']

In [67]:
db.query(f"SELECT * FROM {config.REFRESH_TOKEN_JTI} WHERE jti=='{jti}' and token=='{refresh_token}'")

,jti,token,client_id,exp,active,remark
0,c9e81f25-9144-4dc0-ba87-36a0b556163e,eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJpYXQiO...,5555,1733022589,True,valid


In [68]:
query = f"""UPDATE {config.REFRESH_TOKEN_JTI} SET active = ?, remark = ? WHERE jti=='{jti}' AND token=='{refresh_token}';"""

db.execute(query, (False, "test"))

In [69]:
db.query(f"SELECT * FROM {config.REFRESH_TOKEN_JTI} WHERE jti=='{jti}' and token=='{refresh_token}'")

,jti,token,client_id,exp,active,remark
0,c9e81f25-9144-4dc0-ba87-36a0b556163e,eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJpYXQiO...,5555,1733022589,False,test


In [70]:
signature = sing_signature(
    jti=str(uuid.uuid4()), iat=int(time.time()), exp=config.DPOP_TOKEN_TIME, 
    method="GET", url=f"{BASE_URL}/authorizer/refresh", data={"client_id": "5555"},
    public_key=km.load_public_key_from_pem(),
    private_key=km.load_private_key_from_pem(),
)
response = test_get_endpoint(url=f"{BASE_URL}/authorizer/refresh", client_signature=signature, access_token=refresh_token)
response

status code:  403


{'detail': 'Unauthorize: refresh token replayed.'}

In [71]:
db.query(f"SELECT * FROM {config.REFRESH_TOKEN_JTI}")

,jti,token,client_id,exp,active,remark
0,c9e81f25-9144-4dc0-ba87-36a0b556163e,eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJpYXQiO...,5555,1733022589,False,replayed


## Invalid refresh token: expired

In [72]:
refresh_database(db, config)

In [73]:
client_public = km.load_public_key_from_pem()
client_public_jwk = server.convert_pem_to_jwk(client_public)
thumbprint = server.encode_public_key_jwk_thumprint(client_public_jwk)

iat = int(time.time())
private_key = skm.load_private_key_from_pem()
client_id = "5555"
access_token_jti = str(uuid.uuid4())
exp = iat+1
access_token = server.generate_token(
    jti=access_token_jti,
    iat=iat, 
    exp=exp,
    private_key=private_key,
    algorithm=server.algorithm,
    data={
        "client_id": client_id, 
        "cnf":{"jkt": thumbprint}
    }
)
token_store.add_token(table=config.ACCESS_TOKEN_JTI, jti=access_token_jti, token=access_token, client_id=client_id, exp=exp, active=True, remark="valid")

time.sleep(1)

refresh_token_jti = str(uuid.uuid4())
exp = iat+1
refresh_token = server.generate_token(
    jti=refresh_token_jti,
    iat=iat, 
    exp=exp,
    private_key=private_key,
    algorithm=server.algorithm,
    data={
        "client_id":client_id
    }
)
token_store.add_token(table=config.REFRESH_TOKEN_JTI, jti=refresh_token_jti, token=refresh_token, client_id=client_id, exp=exp, active=True, remark="valid")

In [74]:
signature = sing_signature(
    jti=str(uuid.uuid4()), iat=int(time.time()), exp=config.DPOP_TOKEN_TIME, 
    method="GET", url=f"{BASE_URL}/authorizer/refresh", data={"client_id": "5555"},
    public_key=km.load_public_key_from_pem(),
    private_key=km.load_private_key_from_pem(),
)
response = test_get_endpoint(url=f"{BASE_URL}/authorizer/refresh", client_signature=signature, access_token=refresh_token)
response

status code:  403


{'detail': 'Unauthorize: refresh token expired.'}

In [75]:
db.query(f"SELECT * FROM {config.ACCESS_TOKEN_JTI}")

,jti,token,client_id,exp,active,remark
0,39e053ee-4938-4963-b0ee-cf67f0a21bd5,eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJpYXQiO...,5555,1732936195,True,valid


In [76]:
db.query(f"SELECT * FROM {config.REFRESH_TOKEN_JTI}")

,jti,token,client_id,exp,active,remark
0,561fb3ae-1edf-4c49-b923-780580dd6819,eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJpYXQiO...,5555,1732936195,False,expired


## Invalid refresh token: not exist

In [77]:
refresh_database(db, config)

In [78]:
client_public = km.load_public_key_from_pem()
client_public_jwk = server.convert_pem_to_jwk(client_public)
thumbprint = server.encode_public_key_jwk_thumprint(client_public_jwk)

iat = int(time.time())
private_key = skm.load_private_key_from_pem()
client_id = "5555"

refresh_token_jti = str(uuid.uuid4())
exp = iat+config.REFRESH_TOKEN_TIME
refresh_token = server.generate_token(
    jti=refresh_token_jti,
    iat=iat, 
    exp=exp,
    private_key=private_key,
    algorithm=server.algorithm,
    data={
        "client_id":client_id
    }
)

In [79]:
signature = sing_signature(
    jti=str(uuid.uuid4()), iat=int(time.time()), exp=config.DPOP_TOKEN_TIME, 
    method="GET", url=f"{BASE_URL}/authorizer/refresh", data={"client_id": "5555"},
    public_key=km.load_public_key_from_pem(),
    private_key=km.load_private_key_from_pem(),
)
response = test_get_endpoint(url=f"{BASE_URL}/authorizer/refresh", client_signature=signature, access_token=refresh_token)
response

status code:  400


{'detail': 'refresh token inexist.'}

# Records

In [80]:
db.query(f"SELECT * FROM {config.DPOP_PROOF_JTI}")

,jti


In [81]:
db.query(f"SELECT * FROM {config.ACCESS_TOKEN_JTI}")

,jti,token,client_id,exp,active,remark


In [82]:
db.query(f"SELECT * FROM {config.REFRESH_TOKEN_JTI}")

,jti,token,client_id,exp,active,remark
